In [40]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv('english_news.csv', delimiter='\t', header=0) #non usare delimiter e sep insieme
                                                            
df

,source,date,title,author,text
0,Mainichi Shimbun,2021-12-06,"Editorial: Japan govt's 20,000-yen handout mis...",NaN,The Japanese government is trying to boost the...
1,Mainichi Shimbun,2021-12-07,Editorial: Time for Japan PM Kishida to delive...,NaN,"Prime Minister Fumio Kishida, who won a public..."
2,Mainichi Shimbun,2021-12-08,"Editorial: 80 years on from Pearl Harbor, acti...",NaN,It has been 80 years since the start of the Pa...
3,Mainichi Shimbun,2021-12-08,"Yoroku: 80 years later, many lessons still not...",NaN,"""The world has changed completely. The era was..."
4,Mainichi Shimbun,2021-12-08,Promoting respect for human rights behind US '...,NaN,The U.S. administration under President Joe Bi...
...,...,...,...,...,...
36884,The Japan Times,2021-06-06,COVID-19 tracker: Tokyo reports 351 new cases ...,NaN,Tokyo confirmed 351 new cases of COVID-19 on S...
36885,The Japan Times,2021-06-07,Dollar weakens to around ¥109.40 in Tokyo,NaN,The dollar eased to around ¥109.40 in Tokyo tr...
36886,The Japan Times,2021-06-07,"Nikkei ends higher, but gains limited",NaN,The benchmark Nikkei average turned higher on ...
36887,The Japan Times,2021-06-07,COVID-19 tracker: Osaka reports 72 new cases b...,NaN,Osaka Prefecture reported 72 new COVID-19 case...


In [42]:
df.describe()

,source,date,title,author,text
count,36889,36889,36889,12846,36766
unique,2,1504,36059,1318,35932
top,The Japan Times,2003-06-06,KANSAI: Who & What,Philip Brasor,This could be due to a conflict with your ad-b...
freq,36815,66,23,318,5


In [43]:
df = df.dropna(subset=['source', 'text']) # per ora elimino solo i NaN su date e testi che sono quelli che interessano per la classificazione

In [44]:
df.describe()

,source,date,title,author,text
count,36766,36766,36766,12724,36766
unique,2,1495,35936,1316,35932
top,The Japan Times,2003-06-06,KANSAI: Who & What,Philip Brasor,This could be due to a conflict with your ad-b...
freq,36692,66,23,318,5


In [45]:
# strip di virgole e puntati, solo lettere e spazi e faccio i tetragrammi
# giacomo beve un drink



In [46]:
# Processo di classificazione

X = df['text']
Y = df['date']

# eventualmente fare processo di K-folds cross validation 
# alternativa: Hyperdimensional computing
import torch, torchhd

d = 10000

keys = torchhd.random(3, d)

# country, capital, currency = keys

# inserisco in una lista ogni fonte presente in df
ary_source = np.array([])
for editor in df['source']:
    ary_source = np.append(ary_source, editor)

# elimino i doppioni
ary_source = np.unique(ary_source)

dict_ = {}
for i in ary_source:
    values = torch.stack([])
    us = torchhd.hash_table(keys, )
    dict[i] = [values, us]

KeyboardInterrupt: 